# Recommender System:

- The last thing to do is to use our saved models to recommend items to users:



### For the requested user:
- Calculate the score for every item.
- Sort the items based on the score and output the top results.


### Check which users exist on the test set

In [1]:
import pandas as pd

import pickle
import pandas as pd
import numpy as np

import os

#Keras
from keras.models import load_model
from keras import backend as K

# Tensorflow
import tensorflow as tf

from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


### Set and Check GPUs

In [2]:
def set_check_gpu():
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.per_process_gpu_memory_fraction =1 # allow all of the GPU memory to be allocated
    # for 8 GPUs
    # cfg.gpu_options.visible_device_list = "0,1,2,3,4,5,6,7" # "0,1"
    # for 1 GPU
    cfg.gpu_options.visible_device_list = "0"
    #cfg.gpu_options.allow_growth = True  # # Don't pre-allocate memory; dynamically allocate the memory used on the GPU as-needed
    #cfg.log_device_placement = True  # to log device placement (on which device the operation ran)
    sess = K.tf.Session(config=cfg)
    K.set_session(sess)  # set this TensorFlow session as the default session for Keras

    print("* TF version: ", [tf.__version__, tf.test.is_gpu_available()])
    print("* List of GPU(s): ", tf.config.experimental.list_physical_devices() )
    print("* Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU'))) 
  
    
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
    # set for 8 GPUs
#     os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7";
    # set for 1 GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = "0";

    # Tf debugging option
    tf.debugging.set_log_device_placement(True)

    gpus = tf.config.experimental.list_physical_devices('GPU')

    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

#     print(tf.config.list_logical_devices('GPU'))
    print(tf.config.experimental.list_physical_devices('GPU'))
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [3]:
set_check_gpu()

* TF version:  ['1.15.2', True]
* List of GPU(s):  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
* Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


In [4]:
from sklearn.model_selection import train_test_split

review_data = pd.read_csv('../data/amazon_reviews_us_Shoes_v1_00_help_voted_And_cut_lognTail.csv')
review_data.rename(columns={ 'star_rating': 'score','customer_id': 'user_id', 'user': 'user_name'}, inplace=True)

items = review_data.product_id.unique()
item_map = {i:val for i,val in enumerate(items)}
inverse_item_map = {val:i for i,val in enumerate(items)}
review_data["old_item_id"] = review_data["product_id"] # copying for join with metadata
review_data["item_id"] = review_data["product_id"].map(inverse_item_map)
items = review_data.item_id.unique()
print ("We have %d unique items in metadata "%items.shape[0])

users = review_data.user_id.unique()
user_map = {i:val for i,val in enumerate(users)}
inverse_user_map = {val:i for i,val in enumerate(users)}
review_data["old_user_id"] = review_data["user_id"] 
review_data["user_id"] = review_data["user_id"].map(inverse_user_map)

items_reviewed = review_data.product_id.unique()
review_data["old_item_id"] = review_data["product_id"] # copying for join with metadata
review_data["item_id"] = review_data["product_id"].map(inverse_item_map)

items_reviewed = review_data.item_id.unique()
users = review_data.user_id.unique()
helpful_votes = review_data.helpful_votes.unique()



ratings_train, ratings_test = train_test_split( review_data, test_size=0.1, random_state=0)

We have 97758 unique items in metadata 


In [5]:
ratings_test.user_id.value_counts().sort_values(ascending=False).head(10)

15136    5
15745    4
2378     4
3374     4
47739    4
5981     4
2943     4
36       4
29962    4
1114     4
Name: user_id, dtype: int64

### ** Create a  recommendation example dataset of 100 users from the test set and 100 random items for each and predict recommendations for them

In [6]:
items = review_data.product_id.unique()
item_map = {i:val for i,val in enumerate(items)}
inverse_item_map = {val:i for i,val in enumerate(items)}
review_data["old_item_id"] = review_data["product_id"] # copying for join with metadata
review_data["item_id"] = review_data["product_id"].map(inverse_item_map)
items = review_data.item_id.unique()
print ("We have %d unique items in metadata "%items.shape[0])

We have 97758 unique items in metadata 


In [7]:
# all_info['description'] = all_info['description'].fillna(all_info['title'].fillna('no_data'))
# all_info['title'] = all_info['title'].fillna(all_info['description'].fillna('no_data').apply(str).str[:20])
# all_info['image'] = all_info['image'].fillna('no_data')
# all_info['price'] = pd.to_numeric(all_info['price'],errors="coerce")
# all_info['price'] = all_info['price'].fillna(all_info['price'].median()) 

In [8]:
review_data.head(n=2)

,user_id,product_id,score,product_title,helpful_votes,old_item_id,item_id,old_user_id
0,0,B00FLH88CO,4,Norwegian Fleece Lined Winter Slippers,9,B00FLH88CO,0,50073594
1,0,B00B3HW97Q,5,2 Pairs Foster Grant Womans Versa Polarized Su...,1,B00B3HW97Q,1,50073594


In [9]:
# creating metadata mappings 
titles = review_data['product_title'].unique()
titles_map = {i:val for i,val in enumerate(titles)}
inverse_titles_map = {val:i for i,val in enumerate(titles)}

# price = review_data['price'].unique()
# price_map = {i:val for i,val in enumerate(price)}
# inverse_price_map = {val:i for i,val in enumerate(price)}

# print ("We have %d prices" %price.shape)
print ("We have %d titles" %titles.shape)


# all_info['price_id'] = all_info['price'].map(inverse_price_map)
review_data['title_id'] = review_data['product_title'].map(inverse_titles_map)

# creating dict from 
item2prices = {}
# for val in review_data[['item_id','price_id']].dropna().drop_duplicates().iterrows():
#     item2prices[val[1]["item_id"]] = val[1]["price_id"]

item2titles = {}
# for val in all_info[['item_id','title_id']].dropna().drop_duplicates().iterrows():
#     item2titles[val[1]["item_id"]] = val[1]["title_id"]
        


# populating the rating dataset with item metadata info
# ratings_train["price_id"] = ratings_train["item_id"].map(lambda x : item2prices[x])
# ratings_train["title_id"] = ratings_train["item_id"].map(lambda x : item2titles[x])


# populating the test dataset with item metadata info
# ratings_test["price_id"] = ratings_test["item_id"].map(lambda x : item2prices[x])
# ratings_test["title_id"] = ratings_test["item_id"].map(lambda x : item2titles[x])


We have 51912 titles


In [10]:
# ratings_test = pd.read_parquet('./data/ratings_test.parquet')
# ratings_train = pd.read_parquet('./data/ratings_train.parquet')

In [12]:
review_data.columns

Index(['user_id', 'product_id', 'score', 'product_title', 'helpful_votes',
       'old_item_id', 'item_id', 'old_user_id', 'title_id'],
      dtype='object')

In [13]:
items = review_data.item_id.unique()
df_items = pd.DataFrame(data=items.flatten(),columns=['item_id'])
df_items = pd.merge(df_items,review_data,how='left',left_on=('item_id'),right_on=('item_id'))
df_items= df_items.sample(100)
df_items['key'] = 1
print ("We have %d unique items  "%df_items['item_id'].shape[0])
# df_items= df_items[['item_id', 'description', 'category', 'title', 'title_id', 'price', 'price_id', 'brand', 'key']]
df_items= df_items[['item_id', 'user_id', 'product_id', 'score', 'product_title', 'helpful_votes', 'old_item_id', 'old_user_id', 'title_id', 'key']]
print(df_items.shape)
df_items.head(2)

We have 100 unique items  
(100, 10)


,item_id,user_id,product_id,score,product_title,helpful_votes,old_item_id,old_user_id,title_id,key
84122,44814,55884,B008EQIRL6,3,Aerosoles Women's Betunia Loafer,2,B008EQIRL6,50499277,27422,1
129032,76399,80881,B00ACX0MXU,4,Momo Baby Girls First Walker/Toddler Ribbons &...,1,B00ACX0MXU,33789784,41447,1


In [16]:
users = ratings_test.user_id.unique()
df_users = pd.DataFrame(data=users.flatten(),columns=['user_id'])
df_users = pd.merge(df_users,ratings_test,how='left',left_on=('user_id'),right_on=('user_id'))
df_users= df_users.sample(100)
df_users['key'] = 1
print ("We have %d unique users  "%df_users['user_id'].shape[0])
df_users= df_users[['user_id', 'key']]
print(df_users.shape)
df_users.head(2)

We have 100 unique users  
(100, 2)


,user_id,key
10475,76810,1
12161,38971,1


## Merge users and item and items metadata

In [29]:
df_unseenData= pd.merge(df_users, df_items, on='key')
del df_unseenData['key']
print ("We have %d unique records in the recommendation example dataset "%df_unseenData.shape[0])
print(df_unseenData.shape)
df_unseenData.sample(10)

We have 10000 unique records in the recommendation example dataset 
(10000, 10)


,user_id_x,item_id,user_id_y,product_id,score,product_title,helpful_votes,old_item_id,old_user_id,title_id
5790,70600,51350,61610,B00BHUAFWK,3,Danner Men's Quarry USA 8-Inch Alloy Toe Work ...,3,B00BHUAFWK,14176417,26274
1598,12809,7277,12983,B00KNT5U6W,5,Vionic Women's Motion Venture Active Lace Up,1,B00KNT5U6W,52224836,2894
8581,81932,94348,92743,B005CNXTZ8,5,120686bk Cuffu Online High Quality Women/Girl ...,2,B005CNXTZ8,17308980,49580
8049,8027,35190,46828,B00DY8SKHA,5,Clarks Women's Lexi Dill Wedge Sandal,2,B00DY8SKHA,19759481,2539
7615,81459,75246,13274,B002RT6IOE,5,Penthouse Women's Garbo Mary Jane,7,B002RT6IOE,45769833,13970
3585,14497,62096,70289,B005P1ARE8,1,New Balance Men's M990v3 Running Shoe,1,B005P1ARE8,46629818,1210
5807,71508,14978,16572,B00J0875XS,5,Naturalizer Women's Naja Wedge Pump,1,B00J0875XS,52443411,11489
8990,83966,51350,61610,B00BHUAFWK,3,Danner Men's Quarry USA 8-Inch Alloy Toe Work ...,3,B00BHUAFWK,14176417,26274
9230,12019,12506,20485,B00LF1Z774,5,Vionic Alanis Women's Toe-post Wedge Sandal,6,B00LF1Z774,44503104,9885
106,38971,4558,8382,B007F55UGW,3,HD Vision Anti-Glare Driving Glasses X-Loop w/...,2,B007F55UGW,16737696,4056


In [30]:
df_unseenData.columns

Index(['user_id_x', 'item_id', 'user_id_y', 'product_id', 'score',
       'product_title', 'helpful_votes', 'old_item_id', 'old_user_id',
       'title_id'],
      dtype='object')

In [31]:
from os import listdir
from os.path import isfile, join

mypath = '../models'

onlyfiles = [f.replace('.h5', '') for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['dense_1_Multiply_50_embeddings_4_epochs_dropout',
 'dense_5_Multiply_50_embeddings_10_epochs_dropout',
 'matrix_facto_10_embeddings_100_epochs',
 'dense_5_Meta_Multiply_50_embeddings_10_epochs_dropout',
 'dense_1_Multiply_50_embeddings_100_epochs_dropout']

## Predict the ratings for the items and users in the a recommendation example dataset:

In [32]:
load_path = "../models/"

models =['dense_1_Multiply_50_embeddings_4_epochs_dropout',
 'dense_5_Multiply_50_embeddings_10_epochs_dropout',
 'matrix_facto_10_embeddings_100_epochs',
 'dense_1_Multiply_50_embeddings_100_epochs_dropout']

models_meta = [
'dense_5_Meta_Multiply_50_embeddings_10_epochs_dropout',
]

# for mod in models:
#     model = load_model(load_path+mod+'.h5')
#     df_unseenData['preds_' + mod] = model.predict([df_unseenData['user_id'],
#                                                    df_unseenData['item_id'],
#                                                    df_unseenData['price_id'],
#                                                    df_unseenData['title_id']])

for mod in models:
    model = load_model(load_path+mod+'.h5')
    df_unseenData['preds_' + mod] = model.predict([df_unseenData['user_id_x'],
                                                   df_unseenData['item_id']])


df_unseenData.head(2)

,user_id_x,item_id,user_id_y,product_id,score,product_title,helpful_votes,old_item_id,old_user_id,title_id,preds_dense_1_Multiply_50_embeddings_4_epochs_dropout,preds_dense_5_Multiply_50_embeddings_10_epochs_dropout,preds_matrix_facto_10_embeddings_100_epochs,preds_dense_1_Multiply_50_embeddings_100_epochs_dropout
0,76810,44814,55884,B008EQIRL6,3,Aerosoles Women's Betunia Loafer,2,B008EQIRL6,50499277,27422,4.175709,4.044053,-0.070675,3.867714
1,76810,76399,80881,B00ACX0MXU,4,Momo Baby Girls First Walker/Toddler Ribbons &...,1,B00ACX0MXU,33789784,41447,4.178771,4.043886,0.006576,3.847464


In [33]:
df_unseenData = df_unseenData.drop(columns=['user_id_y'])
df_unseenData = df_unseenData.rename(columns={'user_id_x': 'user_id'})

In [34]:
df_unseenData['user_id'].head(n=2)

0    76810
1    76810
Name: user_id, dtype: int64

In [35]:
df_unseenData.columns

Index(['user_id', 'item_id', 'product_id', 'score', 'product_title',
       'helpful_votes', 'old_item_id', 'old_user_id', 'title_id',
       'preds_dense_1_Multiply_50_embeddings_4_epochs_dropout',
       'preds_dense_5_Multiply_50_embeddings_10_epochs_dropout',
       'preds_matrix_facto_10_embeddings_100_epochs',
       'preds_dense_1_Multiply_50_embeddings_100_epochs_dropout'],
      dtype='object')

## Check which users exist on the example set

In [40]:
df_unseenData.T

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
user_id,76810,76810,76810,76810,76810,76810,76810,76810,76810,76810,...,22750,22750,22750,22750,22750,22750,22750,22750,22750,22750
item_id,44814,76399,73058,37740,32554,90803,4558,14978,35904,94848,...,51350,55873,11566,54610,65006,62879,16104,66417,7277,33743
product_id,B008EQIRL6,B00ACX0MXU,B000A19RKS,B005ET0LSI,B00AZHA69S,B003CC0THW,B007F55UGW,B00J0875XS,B00M1YF404,B003D7K7ZK,...,B00BHUAFWK,B00JQPSGIS,B00DQYPBO2,B006RAHMLW,B0007D8C3W,B002WN3EPQ,B007JT3JWQ,B00ABB00B2,B00KNT5U6W,B00DJDMRH4
score,3,4,2,5,3,5,3,5,4,4,...,3,5,4,5,2,5,5,5,5,4
product_title,Aerosoles Women's Betunia Loafer,Momo Baby Girls First Walker/Toddler Ribbons &...,Mephisto Melchior,Dg Eyewear White High Fashion Gafas De Sol Wom...,Dansko Women's Tilda Clog,Icebug Men's Metro Fashion Walking Boot,HD Vision Anti-Glare Driving Glasses X-Loop w/...,Naturalizer Women's Naja Wedge Pump,"Metal Shoe Horn 8"" Inches - Made in the USA - ...",Robeez Mini Shoez 3D Little Lamb Mary Jane (In...,...,Danner Men's Quarry USA 8-Inch Alloy Toe Work ...,Boho Southwest Sandstone Crossbody Shoulder Bag,Sanuk Women's Yoga Sling 2 Flip Flop,KEEN Women's Akita Mid Boot,Dr. Scholl's Women's Original Exercise Sandal,GBX Men's 167214 Sandal,Report Women's Hilaria Motorcycle Boot,NIKE TRAINING Style# 323655,Vionic Women's Motion Venture Active Lace Up,Zoot Women's W Ultra 7.0 Running Shoe
helpful_votes,2,1,1,2,4,4,2,1,1,1,...,3,3,2,1,4,1,1,2,1,6
old_item_id,B008EQIRL6,B00ACX0MXU,B000A19RKS,B005ET0LSI,B00AZHA69S,B003CC0THW,B007F55UGW,B00J0875XS,B00M1YF404,B003D7K7ZK,...,B00BHUAFWK,B00JQPSGIS,B00DQYPBO2,B006RAHMLW,B0007D8C3W,B002WN3EPQ,B007JT3JWQ,B00ABB00B2,B00KNT5U6W,B00DJDMRH4
old_user_id,50499277,33789784,49089115,38390130,49564557,12020897,16737696,52443411,35845571,15600242,...,14176417,13748676,35732478,46195636,19411213,16916511,15039752,44541651,52224836,16819105
title_id,27422,41447,40008,24021,7260,47652,4056,11489,18714,49878,...,26274,32360,1944,5932,9246,13623,12177,37132,2894,22024
preds_dense_1_Multiply_50_embeddings_4_epochs_dropout,4.17571,4.17877,4.24095,4.11854,4.11497,4.40366,4.11727,4.3722,4.21221,4.25142,...,3.99358,4.02138,4.0253,4.03755,4.00297,4.00099,4.1275,4.08748,4.01346,4.04356


In [41]:
df_unseenData.user_id.value_counts().sort_values(ascending=False).head(5)
df_unseenData[['user_id','preds_dense_5_Multiply_50_embeddings_10_epochs_dropout']].sort_values('preds_dense_5_Multiply_50_embeddings_10_epochs_dropout',ascending=True).head(5)

,user_id,preds_dense_5_Multiply_50_embeddings_10_epochs_dropout
9702,73270,4.043577
5645,66888,4.043620
5634,66888,4.043637
8002,8027,4.043647
3334,2225,4.043657


## A function that will return recommendation list for a given user

In [65]:
df_unseenData.tail(n=3)

,user_id,item_id,product_id,score,product_title,helpful_votes,old_item_id,old_user_id,title_id,preds_dense_1_Multiply_50_embeddings_4_epochs_dropout,preds_dense_5_Multiply_50_embeddings_10_epochs_dropout,preds_matrix_facto_10_embeddings_100_epochs,preds_dense_1_Multiply_50_embeddings_100_epochs_dropout
9997,22750,66417,B00ABB00B2,5,NIKE TRAINING Style# 323655,2,B00ABB00B2,44541651,37132,4.087482,4.043856,-0.024044,3.827760
9998,22750,7277,B00KNT5U6W,5,Vionic Women's Motion Venture Active Lace Up,1,B00KNT5U6W,52224836,2894,4.013463,4.043976,0.004577,3.884584
9999,22750,33743,B00DJDMRH4,4,Zoot Women's W Ultra 7.0 Running Shoe,6,B00DJDMRH4,16819105,22024,4.043557,4.043909,-0.102391,3.834989


In [83]:
load_path = "../models/"

def get_recommendations(userID , model_scr, df_Data):
    if userID not in df_Data['user_id'].values:
        print("\nUser ID not found %d" %userID)
        return userID
       
#     print("\nRecommendations for user id %d Name: %s is:" % (userID, df_Data.loc[df_Data['user_id'] == userID, 'user_name'].values[0]))
   
    df_output=df_Data.loc[df_Data['user_id'] == userID][['item_id','product_title','helpful_votes', model_scr,    
     ]].sort_values(model_scr,ascending=False).set_index('item_id')
#     print(df_output)
    
    df_output.rename(columns={model_scr: 'score'}, inplace=True)
    
    #add ASIN form item_id
#     df_output['product_id'] = df_Data['item_id'].apply(item_map)
    
    return df_output.head(n=10)

### Recommend items to a given user

- Using dense_5_Multiply_50_embeddings_10_epochs_dropout

In [84]:
df_unseenData.columns

Index(['user_id', 'item_id', 'product_id', 'score', 'product_title',
       'helpful_votes', 'old_item_id', 'old_user_id', 'title_id',
       'preds_dense_1_Multiply_50_embeddings_4_epochs_dropout',
       'preds_dense_5_Multiply_50_embeddings_10_epochs_dropout',
       'preds_matrix_facto_10_embeddings_100_epochs',
       'preds_dense_1_Multiply_50_embeddings_100_epochs_dropout'],
      dtype='object')

In [87]:
#######  User ID: 36754                       
df_output = get_recommendations(userID=76810,
                                model_scr='preds_dense_5_Multiply_50_embeddings_10_epochs_dropout', 
                                df_Data=df_unseenData)
# df_output.head(10)
df_output

,product_title,helpful_votes,score
item_id,,,
58557,Aerosoles Women's Army Slip-On Loafer,1,4.045177
44931,FRYE Women's Carson Lug Riding Boot,4,4.044593
32554,Dansko Women's Tilda Clog,4,4.044463
50001,Sorbothane Ultra Graphite Arch Insole,1,4.044458
91077,Sorel Cub Winter Boot (Toddler/Little Kid),2,4.044388
4754,Walstar Women Basic Glitter Mesh Flat Slip on ...,1,4.044334
3731,Sakroots Artist Circle Medium Satchel,2,4.044243
55287,Clarks Women's Roya Kim Sandal,4,4.044231
83874,Orthaheel Women Melissa Slip-On Mule,1,4.044229


## Make predictions for another user using another model:

In [88]:
#######  User ID
user_id = 22750
df_output = get_recommendations(userID=user_id,
                                model_scr='preds_dense_1_Multiply_50_embeddings_4_epochs_dropout',
                                df_Data=df_unseenData)
df_output

,product_title,helpful_votes,score
item_id,,,
16104,Report Women's Hilaria Motorcycle Boot,1,4.127500
90803,Icebug Men's Metro Fashion Walking Boot,4,4.111892
50545,Dr. Scholl's Women's Balance Boot,1,4.101385
14978,Naturalizer Women's Naja Wedge Pump,1,4.099967
62096,New Balance Men's M990v3 Running Shoe,1,4.094049
83874,Orthaheel Women Melissa Slip-On Mule,1,4.092323
50606,Crocs Boys' CC Superman Clog,1,4.088208
66417,NIKE TRAINING Style# 323655,2,4.087482
3339,Clarks Women's Truffle Sandal,2,4.080009


In [89]:
df_output.columns

Index(['product_title', 'helpful_votes', 'score'], dtype='object')

In [90]:
df_output = df_output.reset_index()
df_output['user_id'] = user_id
df_output['asin'] = df_output['item_id'].apply(lambda x : item_map[x])
df_output['url'] = df_output['item_id'].apply(lambda x : 'https://www.amazon.com/dp/'+item_map[x])
df_output = df_output[['user_id','item_id', 'score', 'asin', 'url', 'product_title']]

In [91]:
df_output.head()

,user_id,item_id,score,asin,url,product_title
0,22750,16104,4.127500,B007JT3JWQ,https://www.amazon.com/dp/B007JT3JWQ,Report Women's Hilaria Motorcycle Boot
1,22750,90803,4.111892,B003CC0THW,https://www.amazon.com/dp/B003CC0THW,Icebug Men's Metro Fashion Walking Boot
2,22750,50545,4.101385,B00IYBEOO0,https://www.amazon.com/dp/B00IYBEOO0,Dr. Scholl's Women's Balance Boot
3,22750,14978,4.099967,B00J0875XS,https://www.amazon.com/dp/B00J0875XS,Naturalizer Women's Naja Wedge Pump
4,22750,62096,4.094049,B005P1ARE8,https://www.amazon.com/dp/B005P1ARE8,New Balance Men's M990v3 Running Shoe


### select 100 users from unseen data

In [92]:
all_predicted_df = pd.DataFrame()

for user_id in df_unseenData.sample(n=100, random_state=0)['user_id']:
    print("selected 100 user_id:", user_id)
    df_output = get_recommendations(userID=user_id ,model_scr='preds_dense_1_Multiply_50_embeddings_4_epochs_dropout',df_Data=df_unseenData)
    df_output = df_output.reset_index()
    df_output['user_id'] = user_id
    df_output['asin'] = df_output['item_id'].apply(lambda x : item_map[x])
    df_output['url'] = df_output['item_id'].apply(lambda x : 'https://www.amazon.com/dp/'+item_map[x])
    df_output = df_output[['user_id','item_id', 'score', 'asin', 'url', 'product_title']]
    
    #concat
    all_predicted_df = all_predicted_df.append(df_output)

selected 100 user_id: 15859
selected 100 user_id: 81425
selected 100 user_id: 80259
selected 100 user_id: 17748
selected 100 user_id: 80259
selected 100 user_id: 90783
selected 100 user_id: 93813
selected 100 user_id: 33245
selected 100 user_id: 48071
selected 100 user_id: 698
selected 100 user_id: 41752
selected 100 user_id: 45098
selected 100 user_id: 80259
selected 100 user_id: 56580
selected 100 user_id: 64666
selected 100 user_id: 20403
selected 100 user_id: 3402
selected 100 user_id: 64666
selected 100 user_id: 50895
selected 100 user_id: 59273
selected 100 user_id: 2853
selected 100 user_id: 17748
selected 100 user_id: 5257
selected 100 user_id: 14957
selected 100 user_id: 12137
selected 100 user_id: 19950
selected 100 user_id: 64767
selected 100 user_id: 85944
selected 100 user_id: 55671
selected 100 user_id: 47963
selected 100 user_id: 77354
selected 100 user_id: 33286
selected 100 user_id: 47963
selected 100 user_id: 47963
selected 100 user_id: 15859
selected 100 user_id: 603

In [93]:
all_predicted_df.to_csv('Shoes_for_100_users_per_10_products_prediction.csv', header=True, index=False)